In [8]:
# Mechanics Roster Optimization
# Import required libraries
import pandas as pd
from ortools.linear_solver import pywraplp

# Load data files
print("Loading data files...")

# Load mechanic skills dataset
mechanic_skills_df = pd.read_excel('../data/mechanic_skills_dataset.xlsx')
print("\nMechanic Skills Dataset:")
print(mechanic_skills_df.head())
print(f"\nShape: {mechanic_skills_df.shape}")

# Extract mechanic IDs
mechanics = sorted(mechanic_skills_df['mechanic_id'].unique().tolist())
print(f"\nMechanics: {mechanics}")
print(f"Number of mechanics: {len(mechanics)}")

# Load base aircraft schedule
base_schedule_df = pd.read_excel('../data/base_aircraft_schedule.xlsx')
print("\n\nBase Aircraft Schedule:")
print(base_schedule_df.head())
print(f"\nShape: {base_schedule_df.shape}")

# Extract unique base IDs
bases = sorted(base_schedule_df['base_id'].unique().tolist())
print(f"\nBases: {bases}")
print(f"Number of bases: {len(bases)}")

# Extract periods (groups)
periods = sorted(base_schedule_df['period'].unique().tolist())
print(f"\nPeriods (Groups): {periods}")

# Extract shifts
shifts = sorted(base_schedule_df['shift'].unique().tolist())
print(f"Shifts: {shifts} (1=day, 2=night)")

# Load cost matrix
cost_matrix_df = pd.read_excel('../data/cost_matrix.xlsx')
print("\n\nCost Matrix:")
print(cost_matrix_df.head())
print(f"\nShape: {cost_matrix_df.shape}")

# Map cost columns A, B, C to base IDs
# Assuming A→1, B→2, C→3 (verify from base_schedule)
base_column_mapping = {'A': 1, 'B': 2, 'C': 3}
print(f"\nBase column mapping: {base_column_mapping}")

# Create cost dictionary: cost[mechanic_id, base_id]
cost_dict = {}
for _, row in cost_matrix_df.iterrows():
    mechanic_id = int(row['id'])
    for col, base_id in base_column_mapping.items():
        if col in cost_matrix_df.columns:
            cost_dict[(mechanic_id, base_id)] = float(row[col])

print(f"\nCost dictionary created with {len(cost_dict)} entries")
print("Sample costs:")
for i, (key, val) in enumerate(list(cost_dict.items())[:5]):
    print(f"  Mechanic {key[0]} to Base {key[1]}: {val}")

# Verify data consistency
print("\n\nData Validation:")
print(f"Mechanics in skills dataset: {len(mechanics)}")
print(f"Mechanics in cost matrix: {len(cost_matrix_df)}")
print(f"Bases in schedule: {bases}")
print(f"Bases in cost matrix: {list(base_column_mapping.values())}")

# Check for inspector skill columns
inspector_skill_columns = [col for col in mechanic_skills_df.columns if col.endswith('_inspec')]
print(f"\nInspector skill columns found: {len(inspector_skill_columns)}")
if inspector_skill_columns:
    print(f"  Inspector columns: {inspector_skill_columns[:5]}..." if len(inspector_skill_columns) > 5 else f"  Inspector columns: {inspector_skill_columns}")

# Check for inspector requirements in base schedule
inspector_req_columns = [col for col in base_schedule_df.columns if col.endswith('_inspec')]
print(f"Inspector requirement columns in base schedule: {len(inspector_req_columns)}")
if inspector_req_columns:
    print(f"  Inspector requirement columns: {inspector_req_columns[:5]}..." if len(inspector_req_columns) > 5 else f"  Inspector requirement columns: {inspector_req_columns}")

Loading data files...

Mechanic Skills Dataset:
   mechanic_id  aw139_af  aw139_r  aw139_av  h175_af  h175_r  h175_av  \
0            1         1        1         0        0       0        0   
1            2         0        0         1        0       0        1   
2            3         0        0         1        1       1        1   
3            4         1        1         0        0       0        1   
4            5         0        0         1        0       0        0   

   sk92_af  sk92_r  sk92_av  aw139_af_inspec  aw139_r_inspec  aw139_av_inspec  \
0        1       1        0                1               1                0   
1        0       0        1                0               0                0   
2        0       0        0                0               0                0   
3        0       0        0                0               1                0   
4        1       1        1                0               0                1   

   h175_af_inspec  h175_r_

In [9]:
# Set up OR-Tools solver
print("Setting up OR-Tools solver...")

# Create solver instance (using SCIP for mixed integer programming)
solver = pywraplp.Solver.CreateSolver('SCIP')

if not solver:
    print("SCIP solver not available, using CBC instead")
    solver = pywraplp.Solver.CreateSolver('CBC')

print(f"Solver: {solver.SolverVersion()}")

# Create decision variables: x[mechanic, base, group, shift]
# x[m, b, g, s] = 1 if mechanic m is assigned to base b in group g with shift s
# x[m, b, g, s] = 0 otherwise

print("\nCreating decision variables...")
x = {}

for m in mechanics:
    for b in bases:
        for g in periods:  # periods are the groups
            for s in shifts:
                var_name = f'x_m{m}_b{b}_g{g}_s{s}'
                x[(m, b, g, s)] = solver.IntVar(0, 1, var_name)

print(f"Created {len(x)} binary decision variables")
print(f"Sample variable: {x[(mechanics[0], bases[0], periods[0], shifts[0])]}")

Setting up OR-Tools solver...
Solver: SCIP 10.0.0 [LP solver: SoPlex 8.0.0]

Creating decision variables...
Created 588 binary decision variables
Sample variable: x_m1_b1_g1_s1


In [10]:
# Constraint 1: Each mechanic can work in at most one shift, one group, and one base
# Σ(b∈Bases) Σ(g∈Groups) Σ(s∈Shifts) x[m, b, g, s] ≤ 1  ∀m ∈ Mechanics

print("Adding first constraint: Each mechanic ≤ 1 assignment...")

constraint_count = 0
for m in mechanics:
    constraint = solver.Constraint(0, 1, f'mechanic_{m}_single_assignment')
    for b in bases:
        for g in periods:
            for s in shifts:
                constraint.SetCoefficient(x[(m, b, g, s)], 1)
    constraint_count += 1

print(f"Added {constraint_count} constraints (one per mechanic)")
print(f"Total constraints so far: {solver.NumConstraints()}")

Adding first constraint: Each mechanic ≤ 1 assignment...
Added 49 constraints (one per mechanic)
Total constraints so far: 49


In [11]:
# Constraint 2: Skill Coverage Constraint
# For each (base, period, shift) combination, the pool of assigned mechanics must 
# collectively have all required skills for all aircraft types present at that base
# For each aircraft type a present at (b, p, s):
#   Σ(m ∈ Mechanics with skill a_af) x[m, b, p, s] ≥ 1
#   Σ(m ∈ Mechanics with skill a_r) x[m, b, p, s] ≥ 1
#   Σ(m ∈ Mechanics with skill a_av) x[m, b, p, s] ≥ 1

print("Adding skill coverage constraints...")

# Aircraft types and their corresponding skill columns
aircraft_types = ['aw139', 'h175', 'sk92']
skill_types = ['_af', '_r', '_av']

# Create a mapping of mechanics to their skills for quick lookup
mechanic_skills = {}
mechanic_inspector_skills = {}  # For inspector skills
for _, row in mechanic_skills_df.iterrows():
    m = int(row['mechanic_id'])
    mechanic_skills[m] = {}
    mechanic_inspector_skills[m] = {}
    for aircraft in aircraft_types:
        for skill in skill_types:
            col_name = f"{aircraft}{skill}"
            if col_name in mechanic_skills_df.columns:
                mechanic_skills[m][col_name] = int(row[col_name])
            # Also check for inspector skills
            inspector_col_name = f"{aircraft}{skill}_inspec"
            if inspector_col_name in mechanic_skills_df.columns:
                mechanic_inspector_skills[m][inspector_col_name] = int(row[inspector_col_name])

constraint_count = 0

# Iterate through each row in base_aircraft_schedule
for _, row in base_schedule_df.iterrows():
    base_id = int(row['base_id'])
    period = int(row['period'])
    shift = int(row['shift'])
    
    # Check which aircraft types are present (count > 0)
    for aircraft in aircraft_types:
        if aircraft in base_schedule_df.columns:
            aircraft_count = row[aircraft]
            if aircraft_count > 0:  # Aircraft type is present
                # For each skill type (_af, _r, _av), ensure at least one mechanic with that skill is assigned
                for skill in skill_types:
                    skill_name = f"{aircraft}{skill}"
                    constraint = solver.Constraint(1, solver.infinity(), 
                                                  f'skill_{skill_name}_base{base_id}_period{period}_shift{shift}')
                    
                    # Find all mechanics with this skill
                    for m in mechanics:
                        if mechanic_skills[m].get(skill_name, 0) == 1:
                            constraint.SetCoefficient(x[(m, base_id, period, shift)], 1)
                    
                    constraint_count += 1

print(f"Added {constraint_count} skill coverage constraints")
print(f"Total constraints: {solver.NumConstraints()}")

Adding skill coverage constraints...
Added 66 skill coverage constraints
Total constraints: 115


In [12]:
# Constraint 3: Inspector Coverage Constraint
# For each (base, period, shift) combination, if an inspector skill is required,
# the pool of assigned mechanics must include at least one mechanic with that inspector skill
# For each inspector requirement column in base_schedule_df:
#   If value > 0 at (b, p, s), then:
#     Σ(m ∈ Mechanics with inspector skill) x[m, b, p, s] ≥ 1

print("Adding inspector coverage constraints...")

inspector_constraint_count = 0

# Get all inspector requirement columns from base schedule
inspector_req_columns = [col for col in base_schedule_df.columns if col.endswith('_inspec')]

if inspector_req_columns:
    # Iterate through each row in base_aircraft_schedule
    for _, row in base_schedule_df.iterrows():
        base_id = int(row['base_id'])
        period = int(row['period'])
        shift = int(row['shift'])
        
        # Check each inspector requirement column
        for inspector_col in inspector_req_columns:
            if inspector_col in base_schedule_df.columns:
                inspector_required = row[inspector_col]
                # Check if inspector is required (value > 0)
                if pd.notna(inspector_required) and inspector_required > 0:
                    # Create constraint: at least one mechanic with this inspector skill must be assigned
                    constraint = solver.Constraint(1, solver.infinity(), 
                                                  f'inspector_{inspector_col}_base{base_id}_period{period}_shift{shift}')
                    
                    # Find all mechanics with this inspector skill
                    for m in mechanics:
                        if mechanic_inspector_skills[m].get(inspector_col, 0) == 1:
                            constraint.SetCoefficient(x[(m, base_id, period, shift)], 1)
                    
                    inspector_constraint_count += 1
    
    print(f"Added {inspector_constraint_count} inspector coverage constraints")
    print(f"Total constraints: {solver.NumConstraints()}")
else:
    print("No inspector requirement columns found in base schedule. Skipping inspector constraints.")

Adding inspector coverage constraints...
Added 21 inspector coverage constraints
Total constraints: 136


In [13]:
# Objective Function: Minimize total cost of moving mechanics to bases
# Minimize: Σ(m∈Mechanics) Σ(b∈Bases) Σ(g∈Groups) Σ(s∈Shifts) cost[m, b] * x[m, b, g, s]

print("Creating objective function...")

objective = solver.Objective()

for m in mechanics:
    for b in bases:
        # Get cost for this mechanic-base pair
        cost = cost_dict.get((m, b), 0)
        for g in periods:
            for s in shifts:
                objective.SetCoefficient(x[(m, b, g, s)], cost)

objective.SetMinimization()

print("Objective function created: Minimize total movement cost")
print(f"Number of variables in objective: {len(x)}")
print(f"Model summary:")
print(f"  Variables: {solver.NumVariables()}")
print(f"  Constraints: {solver.NumConstraints()}")

Creating objective function...
Objective function created: Minimize total movement cost
Number of variables in objective: 588
Model summary:
  Variables: 588
  Constraints: 136


In [14]:
# Solve the optimization problem
print("Solving the optimization problem...")
print("This may take a few moments...\n")

status = solver.Solve()

# Check solution status
if status == pywraplp.Solver.OPTIMAL:
    print("✓ Optimal solution found!\n")
    print(f"Optimal total cost: {objective.Value():.2f}\n")
    
    # Display assignments
    print("Mechanic Assignments:")
    print("-" * 80)
    print(f"{'Mechanic':<10} {'Base':<8} {'Group':<8} {'Shift':<8} {'Cost':<10}")
    print("-" * 80)
    
    assignments = []
    total_cost = 0
    for m in mechanics:
        for b in bases:
            for g in periods:
                for s in shifts:
                    if x[(m, b, g, s)].solution_value() > 0.5:  # Binary variable is 1
                        cost = cost_dict.get((m, b), 0)
                        total_cost += cost
                        shift_name = "Day" if s == 1 else "Night"
                        print(f"{m:<10} {b:<8} {g:<8} {shift_name:<8} {cost:<10.2f}")
                        assignments.append({
                            'mechanic_id': m,
                            'base_id': b,
                            'group': g,
                            'shift': s,
                            'shift_name': shift_name,
                            'cost': cost
                        })
    
    print("-" * 80)
    print(f"\nTotal assignments: {len(assignments)}")
    print(f"Total cost: {total_cost:.2f}")
    print(f"\nUnassigned mechanics: {len(mechanics) - len(assignments)}")
    
elif status == pywraplp.Solver.FEASIBLE:
    print("✓ Feasible solution found (may not be optimal)\n")
    print(f"Total cost: {objective.Value():.2f}\n")
elif status == pywraplp.Solver.INFEASIBLE:
    print("✗ Problem is infeasible - no solution exists that satisfies all constraints")
elif status == pywraplp.Solver.UNBOUNDED:
    print("✗ Problem is unbounded")
else:
    print(f"✗ Solver status: {status}")

Solving the optimization problem...
This may take a few moments...

✓ Optimal solution found!

Optimal total cost: 389.00

Mechanic Assignments:
--------------------------------------------------------------------------------
Mechanic   Base     Group    Shift    Cost      
--------------------------------------------------------------------------------
1          1        1        Day      18.00     
2          3        2        Night    10.00     
3          1        1        Day      27.00     
4          2        2        Day      0.00      
5          3        2        Day      18.00     
7          3        1        Day      21.00     
9          2        2        Day      11.00     
10         2        2        Day      35.00     
16         2        1        Night    31.00     
17         1        2        Night    1.00      
19         1        2        Night    18.00     
21         3        2        Day      11.00     
22         2        2        Night    19.00     
25     

In [15]:
# After solving, add this verification cell
print("\n" + "="*80)
print("OPTIMALITY VERIFICATION")
print("="*80)

if status == pywraplp.Solver.OPTIMAL:
    print("✓ Solver Status: OPTIMAL")
    print(f"✓ Objective Value: {objective.Value():.2f}")
    
    # Check if solver provides best bound (lower bound for minimization)
    try:
        best_bound = solver.Objective().BestBound()
        gap = abs(objective.Value() - best_bound)
        gap_percent = (gap / objective.Value()) * 100 if objective.Value() != 0 else 0
        print(f"✓ Best Bound (Lower Bound): {best_bound:.2f}")
        print(f"✓ Optimality Gap: {gap:.6f} ({gap_percent:.6f}%)")
        if gap < 1e-6:
            print("✓ Gap is essentially zero - solution is proven optimal!")
        else:
            print(f"⚠ Gap exists but within tolerance")
    except:
        print("ℹ Best bound not available from this solver")
    
    # Solver statistics
    print(f"\nSolver Statistics:")
    print(f"  Wall time: {solver.wall_time():.2f} ms")
    print(f"  Iterations: {solver.iterations()}")
    print(f"  Nodes: {solver.nodes()}")
    
else:
    print(f"✗ Solver Status: {status}")
    if status == pywraplp.Solver.FEASIBLE:
        print("⚠ Solution is feasible but may not be optimal")
        print("  Consider increasing time limit or checking solver parameters")


OPTIMALITY VERIFICATION
✓ Solver Status: OPTIMAL
✓ Objective Value: 389.00
✓ Best Bound (Lower Bound): 389.00
✓ Optimality Gap: 0.000000 (0.000000%)
✓ Gap is essentially zero - solution is proven optimal!

Solver Statistics:
  Wall time: 71018.00 ms
  Iterations: 4453
  Nodes: 4


In [16]:
# Verify constraints are satisfied
print("\n" + "="*80)
print("CONSTRAINT VERIFICATION")
print("="*80)

# Verify constraint 1: Each mechanic ≤ 1 assignment
violations = []
for m in mechanics:
    total_assignments = sum(x[(m, b, g, s)].solution_value() 
                          for b in bases for g in periods for s in shifts)
    if total_assignments > 1.001:  # Small tolerance for floating point
        violations.append(f"Mechanic {m} has {total_assignments} assignments")
    elif total_assignments < -0.001:
        violations.append(f"Mechanic {m} has negative assignment: {total_assignments}")

if not violations:
    print("✓ Constraint 1: All mechanics have ≤ 1 assignment")
else:
    print("✗ Constraint 1 violations:")
    for v in violations:
        print(f"  {v}")

# Verify skill coverage constraints
skill_violations = []
for _, row in base_schedule_df.iterrows():
    base_id = int(row['base_id'])
    period = int(row['period'])
    shift = int(row['shift'])
    
    for aircraft in ['aw139', 'h175', 'sk92']:
        if aircraft in base_schedule_df.columns and row[aircraft] > 0:
            for skill in ['_af', '_r', '_av']:
                skill_name = f"{aircraft}{skill}"
                assigned_with_skill = sum(
                    x[(m, base_id, period, shift)].solution_value()
                    for m in mechanics
                    if mechanic_skills[m].get(skill_name, 0) == 1
                )
                if assigned_with_skill < 0.999:  # Should be ≥ 1
                    skill_violations.append(
                        f"Base {base_id}, Period {period}, Shift {shift}: "
                        f"Missing {skill_name} (found {assigned_with_skill})"
                    )

if not skill_violations:
    print("✓ Constraint 2: All skill coverage requirements met")
else:
    print("✗ Skill coverage violations:")
    for v in skill_violations[:10]:  # Show first 10
        print(f"  {v}")
    if len(skill_violations) > 10:
        print(f"  ... and {len(skill_violations) - 10} more")

# Verify inspector coverage constraints
inspector_violations = []
inspector_req_columns = [col for col in base_schedule_df.columns if col.endswith('_inspec')]

if inspector_req_columns:
    for _, row in base_schedule_df.iterrows():
        base_id = int(row['base_id'])
        period = int(row['period'])
        shift = int(row['shift'])
        
        for inspector_col in inspector_req_columns:
            if inspector_col in base_schedule_df.columns:
                inspector_required = row[inspector_col]
                if pd.notna(inspector_required) and inspector_required > 0:
                    # Check if at least one mechanic with this inspector skill is assigned
                    assigned_with_inspector_skill = sum(
                        x[(m, base_id, period, shift)].solution_value()
                        for m in mechanics
                        if mechanic_inspector_skills[m].get(inspector_col, 0) == 1
                    )
                    if assigned_with_inspector_skill < 0.999:  # Should be ≥ 1
                        inspector_violations.append(
                            f"Base {base_id}, Period {period}, Shift {shift}: "
                            f"Missing inspector {inspector_col} (found {assigned_with_inspector_skill})"
                        )
    
    if not inspector_violations:
        print("✓ Constraint 3: All inspector coverage requirements met")
    else:
        print("✗ Inspector coverage violations:")
        for v in inspector_violations[:10]:  # Show first 10
            print(f"  {v}")
        if len(inspector_violations) > 10:
            print(f"  ... and {len(inspector_violations) - 10} more")
else:
    print("ℹ Constraint 3: No inspector requirements found (skipped)")

print(f"\n✓ Total constraint violations: {len(violations) + len(skill_violations) + len(inspector_violations)}")


CONSTRAINT VERIFICATION
✓ Constraint 1: All mechanics have ≤ 1 assignment
✓ Constraint 2: All skill coverage requirements met
✓ Constraint 3: All inspector coverage requirements met

✓ Total constraint violations: 0
